In [ ]:
## This code can use existing models and load them with weights created during training - useful for seeing G output
import torch
import torch.nn as nn
import time
import matplotlib.pyplot as plt

LATENT_DIM = 200  # Updated latent dimension
G_IS_512 = 0  # Generator model is trained for 512x512 images

# If G_IS_512 is 1, the generator model is trained for 512x512 images
if G_IS_512:
    IMG_SIZE = 512  # Set to 512x512 for the new output size
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            self.init_size = IMG_SIZE // 32  # Adjusted for 512x512 images (512 / 32 = 16)
            self.l1 = nn.Sequential(nn.Linear(LATENT_DIM, 512 * self.init_size ** 2))
            self.conv_blocks = nn.Sequential(
                nn.BatchNorm2d(512),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(512, 256, 3, stride=1, padding=1),
                nn.BatchNorm2d(256, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(256, 128, 3, stride=1, padding=1),
                nn.BatchNorm2d(128, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(128, 64, 3, stride=1, padding=1),
                nn.BatchNorm2d(64, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(64, 32, 3, stride=1, padding=1),
                nn.BatchNorm2d(32, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(32, 16, 3, stride=1, padding=1),
                nn.BatchNorm2d(16, 0.8),
                nn.ReLU(inplace=True),
                nn.Conv2d(16, 3, 3, stride=1, padding=1),
                nn.Tanh()
            )

        def forward(self, z):
            out = self.l1(z)
            out = out.view(out.shape[0], 512, self.init_size, self.init_size)
            img = self.conv_blocks(out)
            return img
else:
    IMG_SIZE = 256
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            self.init_size = IMG_SIZE // 16  # Adjusted for 256x256 images
            self.l1 = nn.Sequential(nn.Linear(LATENT_DIM, 256 * self.init_size ** 2))
            self.conv_blocks = nn.Sequential(
                nn.BatchNorm2d(256),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(256, 256, 3, stride=1, padding=1),
                nn.BatchNorm2d(256, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(256, 128, 3, stride=1, padding=1),
                nn.BatchNorm2d(128, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(128, 64, 3, stride=1, padding=1),
                nn.BatchNorm2d(64, 0.8),
                nn.ReLU(inplace=True),
                nn.Upsample(scale_factor=2),
                nn.Conv2d(64, 32, 3, stride=1, padding=1),
                nn.BatchNorm2d(32, 0.8),
                nn.ReLU(inplace=True),
                nn.Conv2d(32, 3, 3, stride=1, padding=1),
                nn.Tanh()
            )

        def forward(self, z):
            out = self.l1(z)
            out = out.view(out.shape[0], 256, self.init_size, self.init_size)
            img = self.conv_blocks(out)
            return img
        
def load_generator_weights(generator, weight_path):
    """Load the generator weights from the given file path."""
    generator.load_state_dict(torch.load(weight_path))
    generator.eval()  # Set the generator to evaluation mode

def measure_memory_usage(generator, input_noise, device):
    """Measure the GPU memory usage for generating a single image."""
    torch.cuda.reset_peak_memory_stats(device)  # Reset peak memory stats
    with torch.no_grad():  # No need to track gradients for inference
        _ = generator(input_noise)

    peak_memory = torch.cuda.max_memory_allocated(device)  # Peak memory usage on GPU
    return peak_memory

def measure_execution_time(generator, input_noise):
    """Measure the execution time for generating a single image."""
    start_time = time.time()

    with torch.no_grad():  # No need to track gradients for inference
        _ = generator(input_noise)

    end_time = time.time()
    execution_time = end_time - start_time

    return execution_time

def main():
    # Check if GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load your trained generator model
    generator = Generator().to(device)  # Ensure model is on the GPU

    # Path to your saved generator weights
    weight_path = 'modele/saved_model_dcgan_flower2_256_100.pth'

    # Load weights into the model
    load_generator_weights(generator, weight_path)

    # Define input noise for generator (latent space vector)
    latent_dim = 200  # Updated latent dimension size
    input_noise = torch.randn(1, latent_dim).to(device)  # Batch size of 1 for single image generation

    # Measure memory usage on GPU
    memory_used = measure_memory_usage(generator, input_noise, device)
    print(f"Peak memory used for generating a single image: {memory_used / 1024 ** 2:.2f} MB")

    # Measure execution time
    execution_time = measure_execution_time(generator, input_noise)
    print(f"Execution time for generating a single image: {execution_time:.6f} seconds")

    # Show the generated image
    with torch.no_grad():
        generated_image = generator(input_noise).cpu().squeeze(0).permute(1, 2, 0) / 2.0 + 0.5
        plt.imshow(generated_image)
        plt.axis('off')
        plt.show()
main()